# Performanc Benchmark to PyEMMA
In the following comparison we want to show that the performance of msmhelper is competitive to that of PyEMMA. It should be kept in mind that while both packages focus on the construction and analysis of MSM, PyEMMA has much more functionality and both packages approach Markov State modeling with a different philosophy. Nevertheless, in the following we will compare a few functions that both packages provide. We will ignore the linear algebra methods, since both packages rely on numpy.

In general, performance depends very much on the architecture and Python/package versions. Therefore, it is best to look at the benchmark results on your own device. To do this, you can simply download the Jupyter Notebook from the icon above.

In [1]:
# importing packages needed for benchmark
import msmhelper as mh
import pyemma
import numpy as np

def generate_traj(n_steps, n_states):
    """Generate random state trajectory."""
    return np.random.randint(low=1, high=n_states + 1, size=n_steps)

## Estimation of a Markov State Model

In [2]:
# create random trajectory
n_steps = int(1e5)
n_states = 10
lagtime = 100

Using numba the source code gets compiled just-in-time on the first usage. Hence, we need to run the code once in advance to measure the performance without compilation time. For further informations see the [numba docs](https://numba.readthedocs.io/en/stable/user/5minguide.html#how-to-measure-the-performance-of-numba).

In [3]:
traj = generate_traj(n_steps, n_states),
_ = mh.msm.estimate_markov_model(traj, lagtime=lagtime)
_ = mh.msm.implied_timescales(traj, [lagtime], ntimescales=2)
_ = mh.msm.ck_test(traj, [lagtime], tmax=1000)
_ = mh.msm.timescales.propagate_MCMC(traj, lagtime, 100)

In the following, we compare the determination of the Markov State Model from a numpy state trajectory.

In [4]:
%%timeit -r 10 -n 1 traj = generate_traj(n_steps, n_states)
mh.msm.estimate_markov_model(traj, lagtime=lagtime)

2.62 ms ± 418 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [5]:
%%timeit -r 10 -n 1 traj = generate_traj(n_steps, n_states)
pyemma.msm.estimate_markov_model(traj, lag=lagtime, reversible=False)

7.54 ms ± 1e+03 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [6]:
%%timeit -r 10 -n 1 traj = generate_traj(n_steps, n_states)
pyemma.msm.estimate_markov_model(traj, lag=lagtime)

7.71 ms ± 1.28 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


If you have already formatted the trajectory, msmhelper is even significantly faster again.

In [7]:
%%timeit -r 10 -n 1 traj = mh.StateTraj(generate_traj(n_steps, n_states))
traj.estimate_markov_model(lagtime=lagtime)

339 µs ± 92.8 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)


Increasing the number of states and steps we find:

In [8]:
# create random trajectory with more states and frames
n_steps = int(1e7)
n_states = 100

In [9]:
%%timeit -r 5 -n 1 traj = mh.StateTraj(generate_traj(n_steps, n_states))
traj.estimate_markov_model(lagtime=lagtime)

16.6 ms ± 1.68 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [10]:
%%timeit -r 5 -n 1 traj = generate_traj(n_steps, n_states)
mh.msm.estimate_markov_model(traj, lagtime=lagtime)

357 ms ± 7.12 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [11]:
%%timeit -r 5 -n 1 traj = generate_traj(n_steps, n_states)
pyemma.msm.estimate_markov_model(traj, lag=lagtime, reversible=False)

770 ms ± 28.6 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


## Estimation of Implied Timescales

An important property of Markov State models are the implied time scales. These correspond to the $i$-th eigenvalue $\lambda_i$ of the transition matrix $T_{ij}$
and are defined by
$$t_i = - \frac{t_\text{lag}}{\log(\lambda_i)}$$

In [12]:
# create random trajectory
n_steps = int(1e5)
n_states = 10
n_timescales = 2

# creating lagtimes
lagtimes = np.unique(np.geomspace(1, 100, 20).astype(int))
print(f'lagtimes: {", ".join(lagtimes.astype(str))}')

# catch warnings, because random state trajectory has mainly complex eigenvalues
import warnings

lagtimes: 1, 2, 3, 4, 5, 6, 8, 11, 14, 18, 23, 29, 37, 48, 61, 78, 100


In [13]:
%%timeit -r 5 -n 1 traj = generate_traj(n_steps, n_states)
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    mh.msm.implied_timescales(traj, lagtimes, ntimescales=n_timescales)

16.1 ms ± 6.39 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [14]:
%%timeit -r 5 -n 1 traj = generate_traj(n_steps, n_states)
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    pyemma.msm.its(
        traj,
        lagtimes,
        nits=n_timescales,
        show_progress=False,
        reversible=False,
        n_jobs=1,  # keeping this to None it does not work with PyEMMA 2.5.12
    ).timescales

145 ms ± 15.5 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


## Chapman-Kolmogorov Test
The most important test to check the Markovianity of a MSM is the Chapman-Kolmogorov test which visualizes the agreement of the Chapman-Kolomogorov equation
$$T(\tau n) = T^n(\tau)$$
The following comparison is not easy to interpret, because both packages do not determine the same thing. However, it should be sufficient to get a feeling.

In [15]:
%%timeit -r 5 -n 1 traj = generate_traj(n_steps, n_states)
mh.msm.ck_test(traj, lagtimes, tmax=1000)

38.9 ms ± 1.78 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [16]:
%%timeit -r 5 -n 1 traj = generate_traj(n_steps, n_states)
for lagtime in lagtimes:
    msm = pyemma.msm.estimate_markov_model(traj, lag=lagtime)
    msm.cktest(2, n_jobs=1, show_progress=False)

1.72 s ± 21.6 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


## Propagating a Markov Chain Monte Carlo
We now consider the propagation of a Markov chain Monte Carlo, since this plays a central role in the package msmhelper to estimate the time scales.

In [17]:
# decrease number of steps to speed up msm estimation
n_steps = int(1e4)
n_states = 10

# number to propagate MCMC
n_mcmc_steps = int(1e6)

In [18]:
%%timeit -r 5 -n 1 traj = mh.StateTraj(generate_traj(n_steps, n_states))
mcmc = mh.msm.timescales.propagate_MCMC(traj, lagtime, n_mcmc_steps)

42 ms ± 1.94 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [19]:
%%timeit -r 5 -n 1 msm = pyemma.msm.estimate_markov_model(generate_traj(n_steps, n_states), lag=lagtime)
msm.generate_traj(n_mcmc_steps)

2.96 s ± 63.2 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
